<a href="https://colab.research.google.com/github/okantekeli/imageprocess_final/blob/main/DIP_Final_Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# FINAL

# Kütüphane Yükleme

In [ ]:
# 1. Kütüphanelerin içe aktarılması
import pandas as pd
from skimage import exposure
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 as cv
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
sns.set()
from PIL import Image

In [ ]:
#resimlerin clone edilmesi
!git clone https://github.com/hardik0/AI-for-Medicine-Specialization
%cd AI-for-Medicine-Specialization/AI-for-Medical-Diagnosis/

Cloning into 'AI-for-Medicine-Specialization'...
remote: Enumerating objects: 1601, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 1601 (delta 9), reused 2 (delta 0), pack-reused 1573 (from 1)
Receiving objects: 100% (1601/1601), 720.36 MiB | 30.72 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Updating files: 100% (1497/1497), done.
/content/AI-for-Medicine-Specialization/AI-for-Medical-Diagnosis/AI-for-Medicine-Specialization/AI-for-Medical-Diagnosis


In [ ]:
%ls nih/

densenet.hdf5  images_small/  pretrained_model.h5  test.csv  train-small.csv  valid-small.csv


In [ ]:
# DF'in oluşturulması ve kontrolü
train_df = pd.read_csv("nih/train-small.csv")
print(f'There are {train_df.shape[0]} rows and {train_df.shape[1]} columns in this data frame')
train_df.head()

There are 1000 rows and 16 columns in this data frame


,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00008270_015.png,0,0,0,0,0,0,0,0,0,0,0,8270,0,0,0
1,00029855_001.png,1,0,0,0,1,0,0,0,1,0,0,29855,0,0,0
2,00001297_000.png,0,0,0,0,0,0,0,0,0,0,0,1297,1,0,0
3,00012359_002.png,0,0,0,0,0,0,0,0,0,0,0,12359,0,0,0
4,00017951_001.png,0,0,0,0,0,0,0,0,1,0,0,17951,0,0,0


Have a look at the various columns in this csv file. The file contains the names of chest x-ray images ("Image" column) and the columns filled with ones and zeros identify which diagnoses were given based on each x-ray image.

In [ ]:
# Gereksiz sütünların silinmesi
columns = train_df.keys()
columns = list(columns)
columns.remove('PatientId')


In [ ]:
# Resimleri al
images = train_df['Image'].values

# 9 Adet random resim seç
random_images = [np.random.choice(images) for i in range(9)]

# Resimlerin okunacağı path beli
img_dir = 'nih/images_small/'

print('Seçilen 9 Rastgele Resim')

# Adjust the size of your images
plt.figure(figsize=(20,10))

# Iterate and plot random images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.title(random_images[i])
    plt.imshow(img, cmap='gray')
    plt.axis('off')

# Adjust subplot parameters to give specified padding
plt.tight_layout()

** RASTGELE SECİLMİŞ 9 GÖRÜNTÜNÜN AKCİĞER BÖLGESİ KAYBOLMAYACAK ŞEKİLDE CROP EDİLİYOR

In [ ]:
def crop_image(image_name,x,y,h,w):
  image_path = os.path.join(img_dir, image_name)
  img = cv.imread(image_path)
  crop_img = img[y:y+h, x:x+w]
  #cv2_imshow(crop_img)
  return crop_img

In [ ]:
crp_img_1 = crop_image(random_images[0], 55,50,700,850)
crp_img_2 = crop_image(random_images[1], 55,50,800,850)
crp_img_3 = crop_image(random_images[2], 55,50,700,850)
crp_img_4 = crop_image(random_images[3], 55,50,700,850)
crp_img_5 = crop_image(random_images[4], 55,50,700,900)
crp_img_6 = crop_image(random_images[5], 55,50,700,850)
crp_img_7 = crop_image(random_images[6], 55,50,700,850)
crp_img_8 = crop_image(random_images[7], 55,50,700,900)
crp_img_9 = crop_image(random_images[8], 55,50,750,850)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=3)

images = [crp_img_1, crp_img_2, crp_img_3, crp_img_4, crp_img_5,
          crp_img_6, crp_img_7, crp_img_8, crp_img_9]

for i, img in enumerate(images):
    row = i // 3 + 1
    col = i % 3 + 1
    fig.add_trace(go.Image(z=img), row=row, col=col)

fig.update_layout(height=800, width=800, title_text="Cropped Images")
fig.show()

#Görüntü İşleme Adımları

3. Görüntü İşleme ve İyileştirme (Pre-processing)
  1.	Görüntüleri Akciğer bölgeleri kaybolmayacak şekilde kenarlardan kırpın (Crop),(ilgi alanı dışındaki gürültü bölgelerinden kurtulun
  o	Orijinal ve Crop edilmiş görüntüleri görseleştirin.
  2.	Arasınav ödevi çıktılarını inceleyerek Kontrast Germe (Stretching) Histogram Eşitleme (Equalization) yöntemlerinden uygun gördüğünüzü crop edilmiş görüntüye uygulayın. (YORUM)
  o	Crop edilmişve dönüşüme uğramış  görüntüleri görseleştirin.

  3.	Bir önceki adımın çıktısına gürültü azaltma işlemlerinden birini seçerek uygulayın (örn: Median ve Gaussian Blur).
  o	Görüntüleri görseleştirin


In [ ]:
from skimage import exposure
import cv2  # opencv for python package


def preprocess_image(image):
  '''
  3.1 , 3.2 , 3.3
    Görüntü İşleme ve İyileştirme (Pre-processing)

    Görüntüleri Akciğer bölgeleri kaybolmayacak şekilde kenarlardan kırpın (Crop),(ilgi alanı dışındaki gürültü bölgelerinden kurtulun o Orijinal ve Crop edilmiş görüntüleri görseleştirin.

    Arasınav ödevi çıktılarını inceleyerek Kontrast Germe (Stretching) Histogram Eşitleme (Equalization) yöntemlerinden uygun gördüğünüzü crop edilmiş görüntüye uygulayın. (YORUM) o Crop edilmişve dönüşüme uğramış görüntüleri görseleştirin.

    Bir önceki adımın çıktısına gürültü azaltma işlemlerinden birini seçerek uygulayın (örn: Median ve Gaussian Blur). o Görüntüleri görseleştirin
  '''

  # Calculate minimum and maximum for contrast stretching
  min_val, max_val = image.min(), image.max()

  # Step 1: Apply contrast stretching
  min_val, max_val = image.min(), image.max()
  contrast_stretched = (image - min_val) / (max_val - min_val)  # Normalize to [0, 1]
  contrast_stretched = (contrast_stretched * 255).astype(np.uint8)  # Scale to [0, 255] for visualization

  # Step 2: Apply histogram equalization after contrast stretching
  # Step 2: Apply histogram equalization using skimage
  equalized_image = exposure.equalize_hist(contrast_stretched)  # Returns values in [0, 1]
  equalized_image = (equalized_image * 255).astype(np.uint8)    # Scale to [0, 255]

  # Step 3: Apply gamma correction on the equalized image
  gamma = 1.2  # Adjust gamma value as needed
  gamma_corrected = np.power(equalized_image / 255.0, gamma) * 255
  gamma_corrected = gamma_corrected.astype(np.uint8)

  # Plot original image and histogram
  plt.figure(figsize=(18, 10))
  plt.subplot(2, 4, 1)
  plt.imshow(image, cmap='gray')
  plt.title('Original Image')
  plt.colorbar()
  plt.subplot(2, 4, 5)
  plt.hist(image.ravel(), bins=256, color='gray')
  plt.title("Original Image Histogram")

  # Plot contrast-stretched image and histogram
  plt.subplot(2, 4, 2)
  plt.imshow(contrast_stretched, cmap='gray')
  plt.title('Contrast-Stretched Image')
  plt.colorbar()
  plt.subplot(2, 4, 6)
  plt.hist(contrast_stretched.ravel(), bins=256, color='gray')
  plt.title("Contrast-Stretched Histogram")

  # Plot contrast-equalized image and histogram
  plt.subplot(2, 4, 3)
  plt.imshow(equalized_image, cmap='gray')
  plt.title('Contrast Equalized Image')
  plt.colorbar()
  plt.subplot(2, 4, 7)
  plt.hist(equalized_image.ravel(), bins=256, color='gray')
  plt.title("Equalized Histogram")

  plt.tight_layout()
  plt.show()


  # Apply Median Filtering
  median_filtered = cv2.medianBlur(gamma_corrected, 5)

  # Apply Gaussian Blur
  gaussian_filtered = cv2.GaussianBlur(gamma_corrected, (5, 5), 0)

  # Display Results
  plt.figure(figsize=(12, 6))
  plt.subplot(1, 2, 1), plt.imshow(median_filtered, cmap='gray'), plt.title("Median Filtered Image")
  plt.subplot(1, 2, 2), plt.imshow(gaussian_filtered, cmap='gray'), plt.title("Gaussian Filtered Image")
  plt.show()


3.4 Eşit sayısı belirleme
Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, exposure

def set_threshold_number(image):
  '''
  3.4 Eşit sayısı belirleme
  Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
  '''
  # Görüntüyü NumPy dizisine çevirme
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Convert the grayscale image to a NumPy array
  image_array = np.array(gray_image)

  # Gauss yoğunluk dönüşümü (kontrast artırma)
  p2, p98 = np.percentile(image_array, (2, 98))  # Alt ve üst yüzde 2'lik değerleri al
  adjusted_image = exposure.rescale_intensity(image_array, in_range=(p2, p98))

  # Otsu yöntemi ile eşik değeri belirleme
  threshold_value = filters.threshold_otsu(adjusted_image)

  # Eşikleme işlemi
  binary_image = adjusted_image > threshold_value

  # Sonuçları görselleştirme
  fig, ax = plt.subplots(1, 3, figsize=(18, 6))
  ax[0].imshow(image_array, cmap='gray')
  ax[0].set_title("Orijinal Görüntü")
  ax[0].axis('off')

  ax[1].imshow(adjusted_image, cmap='gray')
  ax[1].set_title("Gauss Yoğunluk Dönüşümü Uygulanmış Görüntü")
  ax[1].axis('off')

  ax[2].imshow(binary_image, cmap='gray')
  ax[2].set_title("Eşiklenmiş İkili Görüntü")
  ax[2].axis('off')

  plt.tight_layout()
  plt.show()

  print(f"Eşik Değeri: {threshold_value}")


4.1 1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold) https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

In [ ]:
from skimage.filters import try_all_threshold
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

def thresholding(image):
  '''
    4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
    https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.
  '''
  gray_crp_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

  thresh = threshold_otsu(gray_crp_img)
  binary = gray_crp_img > thresh

  fig, axes = plt.subplots(ncols=3, figsize=(8, 2.5))
  ax = axes.ravel()
  ax[0] = plt.subplot(1, 3, 1)
  ax[1] = plt.subplot(1, 3, 2)
  ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

  ax[0].imshow(gray_crp_img, cmap=plt.cm.gray)
  ax[0].set_title('Original')
  ax[0].axis('off')

  ax[1].hist(gray_crp_img.ravel(), bins=256)
  ax[1].set_title('Histogram')
  ax[1].axvline(thresh, color='r')

  ax[2].imshow(binary, cmap=plt.cm.gray)
  ax[2].set_title('Thresholded')
  ax[2].axis('off')

  plt.show()

  _, otsu_thresh = cv.threshold(gray_crp_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

  # Belirlenen eşik değeri
  otsu_value = _

  print("Otsu ile belirlenen eşik değeri:", otsu_value)


  fig, ax = try_all_threshold(gray_crp_img, figsize=(10, 8), verbose=False)
  plt.show()

4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)

In [ ]:
def apply_thresholding(image, threshold=128):
  '''
  4.2.	Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
  '''
  #global thresholding
  ret1, th1 = cv.threshold(image, threshold, 255, cv.THRESH_BINARY)

  #otsu threasholding
  ret2, th2 = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

  #otsu thresholding after gaussian filtering
  blur = cv.GaussianBlur(image, (5, 5), 0)
  ret3, th3 = cv.threshold(blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

  images = [image, 0, th1,
            image, 0, th2,
            blur, 0, th3]

  titles = ['Original Image', f'Global Thresholding (v = {threshold})',
              'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
  images = [img, th1, th2, th3]

  for i in range(4):
      plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
      plt.title(titles[i])
      plt.xticks([]),plt.yticks([])
  plt.show()

4.3.1.	Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM)
	Giriş ve Çıkış Görüntülerini görseleştirin


In [ ]:
from skimage.morphology import disk, opening, closing , dilation, erosion

def apply_5_morphological_operations(image):
  '''
  4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin
  '''
  otsu_thresh = threshold_otsu(image)
  binary_otsu = image > otsu_thresh
  binary_image = binary_otsu.astype(np.uint8) * 255
  selem = disk(3)

  dilation_image = dilation(binary_image, selem)
  erosion_image = erosion(binary_image, selem)

  # Açma (Opening) işlemi - Gürültüleri temizler
  opened = opening(binary_image, selem)

  # Kapatma (Closing) işlemi - Delikleri doldurur
  closed = closing(opened, selem)

  # Sonuçları görselleştirme
  fig, ax = plt.subplots(1, 5, figsize=(15, 5))
  ax[0].imshow(binary_image, cmap='gray')
  ax[0].set_title("Binary Görüntü")
  ax[0].axis("off")

  ax[1].imshow(opened, cmap='gray')
  ax[1].set_title("Opening (Açma)")
  ax[1].axis("off")

  ax[2].imshow(closed, cmap='gray')
  ax[2].set_title("Closing (Kapatma)")
  ax[2].axis("off")

  ax[3].imshow(dilation_image, cmap='gray')
  ax[3].set_title("Dilation")
  ax[3].axis("off")

  ax[4].imshow(erosion_image, cmap='gray')
  ax[4].set_title("Erosion")
  ax[4].axis("off")

  plt.show()

  return binary_image, opened, closed, dilation_image, erosion_image


4.3.2 Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

In [ ]:

def connected_component_labeling(image):
  '''
  4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.
  '''
  # 2. Connected Component Labeling (CCL) Uygula
  num_labels, labels , stats, centroids = cv2.connectedComponentsWithStats(image , connectivity=8)


  # 4. Etiketleri Görselleştirmek için Etiket Görüntüsünü Renkli Hale Getir
  # Her bölgeye farklı bir renk atanır
  label_hue = np.uint8(179 * labels / np.max(labels))  # Her etikete renk tonu atanır
  colored_labels = cv2.applyColorMap(label_hue, cv2.COLORMAP_JET)

  # Arka planı siyah yap
  colored_labels[label_hue == 0] = 0

  # 5. Görüntüleri Göster/Plot Et
  plt.figure(figsize=(10, 5))
  plt.subplot(1, 3, 1)
  plt.title("Orijinal Görüntü")
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 3, 2)
  plt.title("Closed Görüntü")
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 3, 3)
  plt.title("CCL Çıktısı")
  plt.imshow(colored_labels)
  plt.axis('off')

  plt.tight_layout()
  plt.show()

  print(f"Bulunan bölge sayısı (etiketler dahil): {num_labels}")

  for i in range(1, num_labels):  # 0 numaralı bileşen arka plandır, onu atlıyoruz
    x, y, w, h, area = stats[i]
    cx, cy = centroids[i]

    print(f"Bölge {i}:")
    print(f" - Alan: {area}")
    print(f" - Merkezi: ({cx:.2f}, {cy:.2f})")
    print(f" - Genişlik: {w}, Yükseklik: {h}")
    print(f" - Aspect Ratio: {w/h:.2f}")
    print(f" - Dairesellik: {(4 * np.pi * area) / (w*h):.2f}")
    print("-" * 30)

4.3.3.	Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
o	(cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)


In [ ]:
from skimage.measure import label, regionprops
from skimage.color import label2rgb

def load_image_and_convert_to_binary(image):
  '''
  4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)
  '''
  # 1. Görüntüyü Yükle ve İkili Görüntüye Çevir

  _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)  # İkili görüntü

  # 2. Bağlantı Bileşen Analizi Yap
  labeled_image = label(binary)

  # 3. Bölgesel Özellikleri Hesapla
  regions = regionprops(labeled_image, intensity_image=image)

  # 4. Özellikleri Print Et
  print("Bölge Özellikleri:")
  for i, region in enumerate(regions, start=1):
      #hepsi yazılmak istenirse buradaki break kaldırmak lazım
      if i > 5:
          break
      print(f"\nBölge {i}:")
      print(f" - Centroid (Ağırlık Merkezi): {region.centroid}")
      print(f" - Area (Alan): {region.area}")
      print(f" - Eccentricity (Eksantriklik): {region.eccentricity}")
      print(f" - Extent (Dikdörtgen Kutusu Oranı): {region.extent}")
      print(f" - Mean Intensity (Yoğunluk Ortalaması): {region.mean_intensity:.2f}")

  # 5. Etiket Görüntüsünü Görselleştir
  colored_labels = label2rgb(labeled_image, image=image, bg_label=0)

  plt.figure(figsize=(10, 5))
  plt.subplot(1, 2, 1)
  plt.title("Orijinal Görüntü")
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 2, 2)
  plt.title("Label Görüntüsü")
  plt.imshow(colored_labels)
  plt.axis('off')

  plt.tight_layout()
  plt.show()

4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
 if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

In [ ]:
from skimage.measure import label, regionprops

def find_lung_regions(image):
    '''
      4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
      Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
      if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
      Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın
    '''

    # 1. Görüntüyü ikili hale getir
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # 2. Bağlantı Bileşenleri ve Özellikleri Hesapla
    labeled_image = label(binary)
    regions = regionprops(labeled_image, intensity_image=image)

    # 3. Öznitelikleri İncele
    region_data = []
    for region in regions:
        region_data.append({
            "label": region.label,
            "area": region.area,
            "centroid": region.centroid,
            "eccentricity": region.eccentricity,
            "extent": region.extent,
        })

    # 4. Alanlara Göre Sırala
    region_data = sorted(region_data, key=lambda x: x['area'], reverse=True)

    # 5. En Büyük İki veya Üç Bölgeyi Değerlendir
    if len(region_data) >= 2:
        # İlk iki büyük bölge
        region1 = region_data[0]
        region2 = region_data[1]

        # Ağırlık merkezlerini kontrol et
        centroid1, centroid2 = region1["centroid"], region2["centroid"]
        if abs(centroid1[1] - centroid2[1]) < 50:  # Yatay yakınlık kontrolü
            return [region1, region2]

        # 3. büyük bölgeyi dahil et
        if len(region_data) > 2:
            region3 = region_data[2]
            return [region1, region2, region3]

    # 6. Belirleyici Öznitelik Bulunamazsa, Otomatik Algoritma
    filtered_regions = [r for r in region_data if r["area"] > 500]  # Alan filtresi


    return filtered_regions


4.3.5	Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin

In [ ]:
def apply_morphological_operations_open_close(image, threshold= 127):
  '''
  Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
  '''
  _, binary = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)


  # Bağlantı bileşen analizi
  labeled_image = label(binary)

  # Seçilen label'ları filtrele (örnek: en büyük 2 bölge)
  regions = regionprops(labeled_image)
  selected_labels = sorted(regions, key=lambda x: x.area, reverse=True)[:2]
  filtered_mask = np.isin(labeled_image, [region.label for region in selected_labels]).astype(np.uint8)

  # Morfolojik işlemler için yapı elemanı oluştur
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Daire şeklinde 5x5 yapı elemanı

  # Açma işlemi
  opened = cv2.morphologyEx(filtered_mask, cv2.MORPH_OPEN, kernel)

  # Kapama işlemi
  closed = cv2.morphologyEx(filtered_mask, cv2.MORPH_CLOSE, kernel)

  # Görselleştirme
  plt.figure(figsize=(12, 6))

  plt.subplot(1, 3, 1)
  plt.title("Orijinal İkili Görüntü")
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 3, 2)
  plt.title("Filtrelenmiş Mask")
  plt.imshow(filtered_mask, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 3, 3)
  plt.title("Kapama İşlemi Sonrası")
  plt.imshow(closed, cmap='gray')
  plt.axis('off')

  plt.tight_layout()
  plt.show()

6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın

In [ ]:
def multiplying_the_original_image_with_the_mask(image, threshold= 127):
  '''
  İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştiren metod
  '''
  _, binary = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)

  # Bağlantı bileşen analizi
  labeled_image = label(binary)

  # Seçilen label'ları filtrele (örnek: en büyük 2 bölge)
  regions = regionprops(labeled_image)
  selected_labels = sorted(regions, key=lambda x: x.area, reverse=True)[:2]
  filtered_mask = np.isin(labeled_image, [region.label for region in selected_labels]).astype(np.uint8)

  # Morfolojik işlemler için yapı elemanı oluştur
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
  closed = cv2.morphologyEx(filtered_mask, cv2.MORPH_CLOSE, kernel)

  # Maske ile orijinal görüntüyü çarp
  masked_image = cv2.bitwise_and(image, image, mask=closed)

  # Görselleştirme
  plt.figure(figsize=(12, 6))

  plt.subplot(1, 3, 1)
  plt.title("Orijinal Görüntü")
  plt.imshow(image, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 3, 2)
  plt.title("Sonuç Maske")
  plt.imshow(closed, cmap='gray')
  plt.axis('off')

  plt.subplot(1, 3, 3)
  plt.title("Maske ile Çarpılmış Görüntü")
  plt.imshow(masked_image, cmap='gray')
  plt.axis('off')

  plt.tight_layout()
  plt.show()

#Resim Analizleri

## Resim 1

In [ ]:
cv2_imshow(crp_img_1)

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_1)

In [ ]:
# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_1)

In [ ]:
#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_1)

In [ ]:
# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_1, cv.COLOR_BGR2GRAY)

In [ ]:
#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=110)

In [ ]:
#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

In [ ]:
#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])


In [ ]:
#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

In [ ]:
# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")


In [ ]:
# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

In [ ]:
# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

## Resim 2

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_2)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_2)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_2)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_2, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=118)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

## Resim 3

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_3)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_3)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_3)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_3, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=90)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

##Resim 4


In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_4)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_4)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_4)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_4, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=106)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

##Resim 5

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_5)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_5)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_5)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_5, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=140)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

##Resim 6

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_6)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_6)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_6)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_6, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=123)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

##Resim 7

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_7)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_7)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_7)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_7, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=112)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

##Resim 8

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_8)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_8)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_8)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_8, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=140)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)

##Resim 9

In [ ]:
# 3.1 , 3.2, 3.3 . Görüntü İşleme ve İyileştirme (Pre-processing)

preprocess_image(crp_img_9)

# 3.4 Eşik Değeri Uygula. Tek eşik değeri kullanacaksanız, Görüntülere Gauss yoğunluk dönüşümü gibi akciğer yoğunluk değerleri ile düşük ve yüksek değerleri ayıracak bir transform uygulayın
set_threshold_number(crp_img_9)

#4.1.	Global Thresholding, Otsu veya skimage all filter kütüphanesini kullanarak (from skimage.filters import try_all_threshold)
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html en az 2 thresholding yöntemi  ile eşik değerlerini belirleyiniz.

thresholding(crp_img_9)

# bundan sonraki işlemlerde gray scale image gerekli olduğu için resmimizi grayscale'e dönüştürüyoruz
gray_crp_img = cv.cvtColor(crp_img_9, cv.COLOR_BGR2GRAY)

#4.2 Eşik değerlerini görüntülere uygulayarak binary görüntüleri görselleştirip uygun thresholding yöntemini i seçiniz. (YORUM)
apply_thresholding(gray_crp_img , threshold=112)

#4.3.1. Uygun morfolojik operatörleri, uygun structural element yapısı ve boyutunu seçerek görüntüye uygulayınız. (YORUM) Giriş ve Çıkış Görüntülerini görseleştirin

st_images = apply_5_morphological_operations(gray_crp_img)

#4.3.2.	Görüntüye bağlantı bileşen analizi (connected component labeling-CCL) (cv2.connectedComponents) uygulayarak olası bölge sayısını print CCL çıktısını plot edin.

connected_component_labeling(st_images[2])

#4.3.3. Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin.
#  o (cv2.connectedComponentsWithStats önerebilirim ama scikit-image’ın regionprops u daha iyi)

load_image_and_convert_to_binary(st_images[2])

# 4.3.4  Elde ettiğiniz stats ve labelsları inceleyerek akciğer alanlarını en iyi ifade eden öznitelikleri yorumlayınız.
# Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge,
# if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
# Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

lung_regions = find_lung_regions(st_images[2])

 # Sonuçları Yazdır
for i, region in enumerate(lung_regions, start=1):
        print(f"Bölge {i}:")
        print(f" - Label: {region['label']}")
        print(f" - Area: {region['area']}")
        print(f" - Centroid: {region['centroid']}")
        print(f" - Eccentricity: {region['eccentricity']}")
        print(f" - Extent: {region['extent']}")

# 4.3.5 Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin
apply_morphological_operations_open_close(st_images[2])

# 6. İşlemler sonucunda elde ettiğiniz maske ile orijinal görüntüyü çarparak görüntü işleme sonuçlarını görselleştirip yorumlayın
multiplying_the_original_image_with_the_mask(gray_crp_img)